幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/corrMap/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/corrMap/')
CorrMapGraphPath=StorePath+'/'+TITLE+DATE+'graph/corrMap/'

if not os.path.exists(StorePath+'/'+TITLE+'feature/'): 
    os.makedirs(StorePath+'/'+TITLE+'feature/')
FeaturePath=StorePath+'/'+TITLE+'feature/'

センサデータの読み込み

In [3]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [4]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

%matplotlib inline

In [5]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [6]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [7]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

# Methematical/Statistical Functions

## Mean, Median

In [8]:
def FastVector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].mean() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Mean==========='
    return Output

In [9]:
def FastVector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [np.median(wind[i,:]) for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Median==========='
    return Output

## Variance, Std Deviation

In [10]:
def FastVector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].var() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Variance==========='
    return Output

In [11]:
def FastVector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)  
    Output = np.array( [wind[i,:].var()**0.5 for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of standard Deviation==========='
    return Output

## Min, Max, Range
Range = the difference between maximum and minimum sample values

In [12]:
def FastVector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].max() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Max==========='
    return Output

In [13]:
def FastVector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].min() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Min==========='
    return Output

In [14]:
def FastVector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [(wind[i,:].max()-wind[i,:].min())\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Range==========='
    return Output

## RMS

In [15]:
def FastVector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [((np.sum( wind[i,:]**2)/wind[i,:].size)**0.5)\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Root Means Square==========='
    return Output

## Correlation, Cross-Correlation

In [16]:
def FastVector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)
    Output = np.array( [( np.corrcoef(windX[i,:],windY[i,:]) )\
                    for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Correlation==========='
    return Output[:,0,1]

# Signal magnitude area

In [17]:
def FastVector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    Output = np.array( [(np.sum((windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2)**0.5))\
                for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Signal Magnitude Area==========='
    return Output  

# 各センサの相関を求める

In [21]:
ls /media/takeyama/HD-PNFU3/01_ActivityResearchData/Participants01_First_1/feature/

Label_Sensor1.npz
LeftHand_AccX_Win=0256_Sld=0008_Corr.npz
LeftHand_AccX_Win=0256_Sld=0008_Energy.npz
LeftHand_AccX_Win=0256_Sld=0008_Entropy.npz
LeftHand_AccX_Win=0256_Sld=0008_Frequency.npz
LeftHand_AccX_Win=0256_Sld=0008_Max.npz
LeftHand_AccX_Win=0256_Sld=0008_Mean.npz
LeftHand_AccX_Win=0256_Sld=0008_Median.npz
LeftHand_AccX_Win=0256_Sld=0008_Min.npz
LeftHand_AccX_Win=0256_Sld=0008_RMS.npz
LeftHand_AccX_Win=0256_Sld=0008_SMA.npz
LeftHand_AccX_Win=0256_Sld=0008_StdDev.npz
w=0256_s=0032_CorrelationMapAccX.npz
w=0256_s=0032_JSDMapAccX.npz
w=0256_s=0032_KLDMapAccX.npz


In [22]:
LabelAndData = np.load('/media/takeyama/HD-PNFU3/01_ActivityResearchData/Participants01_First_1/feature/'+'Label_Sensor1.npz')['data'][()]

In [23]:
LabelAndData

array([[-1.0156, -1.0073, -1.0041, ..., -0.07390000000000001, -0.0747,
        -0.0742],
       [-0.042300000000000004, -0.0516, -0.0577, ...,
        -0.024800000000000003, -0.0275, -0.024800000000000003],
       [-0.2341, -0.2341, -0.2409, ..., 0.9472, 0.9514, 0.9502],
       ..., 
       [nan, nan, nan, ..., u'\u306a\u3057', u'\u306a\u3057',
        u'\u306a\u3057'],
       [nan, nan, nan, ..., u'\u76f4\u7acb', u'\u76f4\u7acb',
        u'\u76f4\u7acb'],
       [nan, nan, nan, ..., 114.0, 114.0, 114.0]], dtype=object)

In [24]:
LabelAndData

array([[u'\u505c\u6b62', u'\u505c\u6b62', u'\u505c\u6b62', ...,
        u'\u306a\u3057', u'\u306a\u3057', u'\u306a\u3057'],
       [nan, nan, nan, ..., u'\u306a\u3057', u'\u306a\u3057',
        u'\u306a\u3057'],
       [nan, nan, nan, ..., u'\u306a\u3057', u'\u306a\u3057',
        u'\u306a\u3057'],
       ..., 
       [nan, nan, nan, ..., u'\u306a\u3057', u'\u306a\u3057',
        u'\u306a\u3057'],
       [nan, nan, nan, ..., u'\u76f4\u7acb', u'\u76f4\u7acb',
        u'\u76f4\u7acb'],
       [nan, nan, nan, ..., 114.0, 114.0, 114.0]], dtype=object)